# pag 378 Algoritmos de ensamble

Algoritmos de Conjunto


se tienen distintos modelos y cada unoi da su opinion, y luegop se ¿vota? para ver cual es la mejor respuesta?

### ✅ ¿Qué son los **Algoritmos de Conjunto** (Ensemble Learning)?

Son **técnicas de aprendizaje automático** que combinan **varios modelos individuales** (por ejemplo, varios árboles de decisión, o varios clasificadores) para obtener **una predicción final más precisa y robusta**.

La idea es que **muchos modelos simples, cuando trabajan juntos, pueden hacerlo mejor que un solo modelo complejo.**

---

### 🤖 ¿Cómo funciona eso de que "cada modelo da su opinión"?

Supongamos que queremos saber si una imagen es de un **gato** o de un **perro**. Tenemos 3 modelos distintos:

* **Modelo 1** dice: "Es un gato"
* **Modelo 2** dice: "Es un perro"
* **Modelo 3** dice: "Es un gato"

Aquí, cada modelo da **su opinión o predicción**.

---

### 🗳️ ¿Y cómo se elige la respuesta final? (¿"votar"?)

¡Exactamente! Lo que se hace es una **votación** (en modelos de clasificación). Existen varias formas de combinar las opiniones, pero las más comunes son:

#### 1. **Votación mayoritaria (Majority Voting)**

Se elige la opción que recibió más votos.

> En el ejemplo anterior, 2 modelos dijeron “gato” y 1 dijo “perro”, así que la respuesta final es: **gato**.

#### 2. **Promedio (en regresión)**

Cuando se trabaja con valores numéricos (regresión), se puede hacer el **promedio de las salidas** de los modelos.

---

### 🎯 ¿Por qué es útil usar algoritmos de conjunto?

* **Reduce el error**: Disminuye el riesgo de que un mal modelo arruine la predicción.
* **Mejora la generalización**: Se comporta mejor con datos nuevos.
* **Disminuye el sobreajuste** (cuando un modelo aprende demasiado bien los datos de entrenamiento y falla en los nuevos).

---

### 🧠 Ejemplos conocidos de algoritmos de conjunto:

| Técnica      | Descripción breve                                                                                                  |
| ------------ | ------------------------------------------------------------------------------------------------------------------ |
| **Bagging**  | Usa muchos modelos entrenados con diferentes subconjuntos del conjunto de datos. Ejemplo: **Random Forest**.       |
| **Boosting** | Los modelos se entrenan secuencialmente, corrigiendo los errores del anterior. Ejemplo: **AdaBoost**, **XGBoost**. |
| **Stacking** | Combina las predicciones de varios modelos mediante otro modelo que aprende a combinar sus salidas.                |

---

### 📝 En resumen

Sí, en cierto modo **cada modelo "da su opinión"**, y luego **se combinan todas esas opiniones** (como una votación o promedio) para tomar una decisión final. Esto permite que el sistema **sea más confiable** que depender solo de un modelo.



En los algoritmos de conjunto, como `VotingClassifier`, existen dos formas principales de combinar los resultados de varios modelos:

---

## 🔷 1. **Votación Dura (Hard Voting)**

👉 Cada modelo **vota por una clase** (como si dijera "yo creo que es esto") y gana la clase que **reciba más votos**.

### 🗳️ Ejemplo:

Supongamos que tenemos 3 modelos que predicen si un pasajero sobrevivió (`1`) o no (`0`) en el Titanic:

* Modelo A: **1**
* Modelo B: **0**
* Modelo C: **1**

✅ Resultado final (por mayoría): **1**

**No importa qué tan confiado esté cada modelo**, solo cuenta su voto.

### 📌 Características:

* Sencillo y rápido.
* Funciona bien si todos los modelos tienen una precisión razonable.
* No considera la “confianza” del modelo en su predicción.

---

## 🔷 2. **Votación Suave o Blanda (Soft Voting)**

👉 Cada modelo no solo da su predicción, sino también **la probabilidad** de cada clase. Se suman esas probabilidades y se elige la clase con **mayor suma total**.

### 🎲 Ejemplo:

Supongamos que tres modelos devuelven estas probabilidades para cada clase:

| Modelo | Probabilidad de 0 | Probabilidad de 1 |
| ------ | ----------------- | ----------------- |
| A      | 0.3               | 0.7               |
| B      | 0.6               | 0.4               |
| C      | 0.2               | 0.8               |

🔍 Suma de probabilidades:

* Clase 0: **0.3 + 0.6 + 0.2 = 1.1**
* Clase 1: **0.7 + 0.4 + 0.8 = 1.9**

✅ Resultado final: **1** (tiene mayor probabilidad total)

### 📌 Características:

* Más precisa si los modelos pueden calcular probabilidades (como regresión logística).
* Usa información más rica (la confianza del modelo).
* Requiere que todos los modelos tengan el método `.predict_proba()`.

---

### ⚙️ ¿Cómo se configura esto en Python con `VotingClassifier`?

```python
# Hard voting (por defecto)
VotingClassifier(estimators=[...], voting='hard')

# Soft voting (requiere modelos que tengan predict_proba)
VotingClassifier(estimators=[...], voting='soft')
```

---

### ✅ ¿Cuál usar?

| Situación                                | Recomendación          |
| ---------------------------------------- | ---------------------- |
| Modelos que no tienen `.predict_proba()` | Usa **votación dura**  |
| Modelos con buenas probabilidades        | Usa **votación suave** |
| Buscas algo rápido y sencillo            | Dura                   |
| Quieres precisión más fina               | Suave                  |

---

### Ejemplo con Titanic!!

In [1]:
!pip install seaborn

In [15]:
# Paso 1: Importar librerías
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Paso 2: Importar modelos
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import GaussianNB

# Modelos para ejemplo Bikes
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor, VotingRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.datasets import fetch_openml
from math import sqrt

# Modelos para ejemplo california_housing
from sklearn.datasets import fetch_california_housing



In [3]:
# Paso 3: Cargar dataset del Titanic desde seaborn
titanic = sns.load_dataset("titanic")

# Paso 4: Preprocesamiento simple
# Seleccionamos algunas columnas relevantes
df = titanic[["survived", "pclass", "sex", "age", "fare", "embarked"]].copy()

# Eliminamos filas con valores nulos en 'embarked'
df = df.dropna(subset=["embarked"])

# Imputamos valores faltantes de 'age' con la media
imputer = SimpleImputer(strategy="mean")
df["age"] = imputer.fit_transform(df[["age"]])

# Codificamos las variables categóricas
df["sex"] = LabelEncoder().fit_transform(df["sex"])
df["embarked"] = LabelEncoder().fit_transform(df["embarked"])

# Paso 5: Dividir en X (características) e y (etiqueta)
X = df.drop("survived", axis=1)
y = df["survived"]

In [4]:
# Dividir en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:
# Paso 6: Crear modelos individuales
model_knn = KNeighborsClassifier(n_neighbors=5)
model_tree = DecisionTreeClassifier(random_state=42)
model_logreg = LogisticRegression(max_iter=1000)

In [6]:
# Paso 7: Crear el modelo de conjunto (votación)
voting_clf = VotingClassifier(
    estimators=[
        ('knn', model_knn),
        ('tree', model_tree),
        ('logreg', model_logreg)
    ],
    voting='hard'  # votación mayoritaria
)

# Entrenar el modelo
voting_clf.fit(X_train, y_train)

# Predecir y evaluar
y_pred = voting_clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(f"Precisión del modelo de conjunto (VotingClassifier): {accuracy:.4f}")

Precisión del modelo de conjunto (VotingClassifier): 0.8034


In [8]:
# Ahora con SOFT

# Crear modelos que sí permiten .predict_proba()
model_logreg = LogisticRegression(max_iter=1000)
model_tree = DecisionTreeClassifier(random_state=42)
model_nb = GaussianNB()

In [9]:


# VotingClassifier con votación SUAVE (soft)
voting_soft = VotingClassifier(
    estimators=[
        ('logreg', model_logreg),
        ('tree', model_tree),
        ('nb', model_nb)
    ],
    voting='soft'  # <--- aqui se cambia a suave
)

# Entrenar y evaluar
voting_soft.fit(X_train, y_train)
y_pred_soft = voting_soft.predict(X_test)
accuracy_soft = accuracy_score(y_test, y_pred_soft)

print(f"Precisión con votación SUAVE: {accuracy_soft:.4f}")

Precisión con votación SUAVE: 0.8146


In [10]:
# Comparación con votación DURA
voting_hard = VotingClassifier(
    estimators=[
        ('logreg', model_logreg),
        ('tree', model_tree),
        ('nb', model_nb)
    ],
    voting='hard'  # Aquí usamos votación dura
)

voting_hard.fit(X_train, y_train)
y_pred_hard = voting_hard.predict(X_test)
accuracy_hard = accuracy_score(y_test, y_pred_hard)

print(f"Precisión con votación DURA:  {accuracy_hard:.4f}")
print(f"Precisión con votación SUAVE: {accuracy_soft:.4f}")


Precisión con votación DURA:  0.7809
Precisión con votación SUAVE: 0.8146


### Ejemplo con Bikes/Bicicletas!!

en este se le agregaran pesos a los modelos en votingRegressor

In [12]:
# Cargar datos
bike = fetch_openml(name="Bike_Sharing_Demand", version=2, as_frame=True)
df = bike.frame

# Preprocesamiento
df['holiday'] = df['holiday'].astype(str).str.lower().map({'false': 0, 'true': 1})
df['workingday'] = df['workingday'].astype(str).str.lower().map({'false': 0, 'true': 1})
df['weekday'] = df['weekday'].astype(int)

le_season = LabelEncoder()
le_weather = LabelEncoder()
df['season'] = le_season.fit_transform(df['season'])
df['weather'] = le_weather.fit_transform(df['weather'])

# Variables predictoras y objetivo
X = df[['season', 'hour', 'holiday', 'weekday', 'workingday', 'weather', 'temp', 'feel_temp', 'humidity', 'windspeed']]
y = df['count']

# Escalado
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# División de datos
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.25, random_state=42)



In [13]:
# Modelos individuales
modelos = {
    "Regresión Lineal": LinearRegression(),
    "Árbol de Decisión": DecisionTreeRegressor(random_state=42),
    "KNN": KNeighborsRegressor(n_neighbors=5),
    "SVR": SVR(),
    "SVM (lineal)": SVR(kernel='linear')
}

# Entrenar y evaluar modelos individuales
for nombre, modelo in modelos.items():
    modelo.fit(X_train, y_train)
    y_pred = modelo.predict(X_test)
    print(f"\nModelo: {nombre}")
    print(f"R²: {r2_score(y_test, y_pred):.3f}")
    print(f"RMSE: {sqrt(mean_squared_error(y_test, y_pred)):.2f}")

# Voting Regressor con pesos
voting = VotingRegressor([
    ('lr', LinearRegression()),
    ('rf', RandomForestRegressor(n_estimators=100, random_state=42)),
    ('knn', KNeighborsRegressor(n_neighbors=5))
], weights=[2,3,1])

voting.fit(X_train, y_train)
y_pred_voting = voting.predict(X_test)
print(f"\nModelo: Voting Regressor")
print(f"R²: {r2_score(y_test, y_pred_voting):.3f}")
print(f"RMSE: {sqrt(mean_squared_error(y_test, y_pred_voting)):.2f}")



Modelo: Regresión Lineal
R²: 0.349
RMSE: 144.69

Modelo: Árbol de Decisión
R²: 0.716
RMSE: 95.51

Modelo: KNN
R²: 0.630
RMSE: 109.02

Modelo: SVR
R²: 0.353
RMSE: 144.19

Modelo: SVM (lineal)
R²: 0.305
RMSE: 149.51

Modelo: Voting Regressor
R²: 0.775
RMSE: 85.11


### Ejemplo california_housing

In [17]:
# Cargar datos
data = fetch_california_housing()
df = pd.DataFrame(data.data, columns=data.feature_names)

# Variables predictoras y objetivo
X = df[['MedInc', 'AveRooms']]
y = data.target

# División de datos
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# Escalado
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Modelos individuales
lr = LinearRegression().fit(X_train, y_train)
tree = DecisionTreeRegressor(max_depth=5).fit(X_train, y_train)
knn = KNeighborsRegressor(n_neighbors=5).fit(X_train_scaled, y_train)
svm = SVR().fit(X_train_scaled, y_train)

# Crear muestra para predicción
sample = pd.DataFrame([{
    'MedInc': 3.5,
    'AveRooms': 4
}])

# Predicciones individuales
pred_lr = lr.predict(sample)
pred_tree = tree.predict(sample)
pred_knn = knn.predict(scaler.transform(sample))
pred_svm = svm.predict(scaler.transform(sample))

print("Regresión Lineal:", round(pred_lr[0] * 100000, 2), "USD")
print("Árbol de Decisión:", round(pred_tree[0] * 100000, 2), "USD")
print("KNN:", round(pred_knn[0] * 100000, 2), "USD")
print("SVR:", round(pred_svm[0] * 100000, 2), "USD")

# === VotingRegressor con mismos modelos con pesos ===
voting = VotingRegressor([
    ('lr', LinearRegression()),
    ('tree', DecisionTreeRegressor(max_depth=5)),
    ('knn', KNeighborsRegressor(n_neighbors=5)),
    ('svr', SVR())
], weights=[2,2,1,3])

# Entrenamiento con datos escalados
voting.fit(X_train_scaled, y_train)

# Predicción para muestra
pred_voting = voting.predict(scaler.transform(sample))
print("Voting Regressor:", round(pred_voting[0] * 100000, 2), "USD")

# Evaluación VotingRegressor
y_pred_voting = voting.predict(X_test_scaled)
print("\nEvaluación Voting Regressor:")
print(f"R²: {r2_score(y_test, y_pred_voting):.3f}")
print(f"RMSE: {np.sqrt(mean_squared_error(y_test, y_pred_voting)):.2f}")


Regresión Lineal: 196104.97 USD
Árbol de Decisión: 232202.21 USD
KNN: 292800.2 USD
SVR: 221607.22 USD
Voting Regressor: 226779.53 USD

Evaluación Voting Regressor:
R²: 0.529
RMSE: 0.79
